In [7]:
import pandas as pd
import numpy as np

# Read the DataFrames from Excel files
products_df = pd.read_excel('products_df.xlsx')
parts_supp_df = pd.read_excel('parts_supp_df.xlsx')
products_priority_df = pd.read_excel('products_priority_df.xlsx')

# Convert all columns in products_df to integers
for column in products_df.columns:
    # Check if the data can be converted to integers
    if pd.api.types.is_numeric_dtype(products_df[column]):
        products_df[column] = products_df[column].astype(int)
    else:
        print(f"Warning: Column '{column}' in products_df is not numeric and was not converted.")

# Convert data types explicitly
parts_supp_df['Supply'] = parts_supp_df['Supply'].astype(int)
products_priority_df['Priority Score'] = products_priority_df['Priority Score'].astype(int)


In [8]:
products_df

,Product Code,Demand,Part_1,Part_2,Part_3,Part_4,Part_5,Part_6,Part_7,Part_8,Part_9,Part_10,Part_11,Part_12
0,FAP0001,974,6,8,8,8,6,5,5,6,5,6,8,6
1,FAP0002,634,5,5,5,6,9,6,7,6,6,9,9,5
2,FAP0003,860,7,8,5,5,8,7,7,5,6,7,7,9
3,FAP0004,924,6,9,5,8,6,5,6,6,5,9,7,6
4,FAP0005,538,6,9,6,6,6,5,8,7,9,6,5,8
5,FAP0006,965,5,7,6,9,5,9,6,8,7,5,9,6
6,FAP0007,853,5,7,8,5,6,9,8,8,8,8,7,8
7,FAP0008,892,8,7,6,5,6,8,6,6,8,6,5,8
8,FAP0009,831,6,9,6,9,9,6,6,7,6,8,8,8
9,FAP0010,950,7,9,7,5,6,5,6,6,5,7,8,6


In [10]:
products_priority_df

,Product Code,Priority Score
0,FAP0001,3
1,FAP0002,4
2,FAP0003,4
3,FAP0004,4
4,FAP0005,4
5,FAP0006,4
6,FAP0007,1
7,FAP0008,2
8,FAP0009,1
9,FAP0010,1


In [11]:
parts_supp_df

,Part_Num,Supply
0,Part_1,37056
1,Part_2,33214
2,Part_3,33393
3,Part_4,31475
4,Part_5,30406
5,Part_6,30989
6,Part_7,33995
7,Part_8,32215
8,Part_9,30425
9,Part_10,38697


In [14]:

def optimize_production(products_df, parts_supp_df, products_priority_df):
    # Merge data on product codes to bring all relevant data together
    merged_df = products_df.merge(products_priority_df, on='Product Code')
    merged_df = merged_df.sort_values('Priority Score', ascending=False)

    # Create a dictionary from the parts supply DataFrame for easy access
    parts_supply = parts_supp_df.set_index('Part_Num')['Supply'].to_dict()

    # Group products by priority score
    grouped_products = merged_df.groupby('Priority Score', as_index=False)

    # Prepare the output list
    output_data = []

    # Process each priority group
    for _, group in grouped_products:
        # Initialize a dictionary to store potential productions
        potential_productions = {}

        # First pass: determine potential maximum production without allocation
        for _, row in group.iterrows():
            product_code = row['Product Code']
            min_possible = row['Demand']
            
            # Calculate potential maximum production based on parts availability
            for part, required in row.iteritems():
                if part.startswith('Part_') and required > 0 and part in parts_supply:
                    if parts_supply[part] > 0:
                        max_product_from_part = parts_supply[part] // required
                        min_possible = min(min_possible, max_product_from_part)
            
            potential_productions[product_code] = min_possible
        
        # Determine the minimum production that could be evenly achieved
        min_production = min(potential_productions.values())

        # Second pass: allocate parts to achieve as close to even backorder percentage as possible
        for _, row in group.iterrows():
            product_code = row['Product Code']
            demand = row['Demand']
            allocated_production = min(min_production, demand)

            # Deduct the parts from the supply
            for part, required in row.iteritems():
                if part.startswith('Part_') and required > 0 and part in parts_supply:
                    parts_supply[part] -= required * allocated_production
            
            # Calculate backorder
            backorder = demand - allocated_production

            # Collect the production data, remaining parts, and backorder
            product_data = {
                'Product Code': product_code,
                'Optimized Production': allocated_production,
                'Backorder': backorder
            }
            for part, remaining in parts_supply.items():
                product_data[part + ' Remaining'] = remaining

            output_data.append(product_data)

    # Create the output DataFrame from the list
    output_df = pd.DataFrame(output_data)

    return output_df

# Example function call
optimized_df = optimize_production(products_df, parts_supp_df, products_priority_df)
print(optimized_df)

import openpyxl
import os
import pandas as pd

# Assuming 'optimized_df' is your DataFrame

# # Specify the directory name where you want to save the file
# directory = 'msn'

# # Check if the directory exists, and if not, create it
# if not os.path.exists(directory):
#     os.makedirs(directory)

# # Specify the path to the Excel file
# file_path = os.path.join(directory, 'optimized_production.xlsx')

# Save the DataFrame to an Excel file
optimized_df.to_excel('optimized_production.xlsx', index=False, engine='openpyxl')

  Product Code  Optimized Production  Backorder  Part_1 Remaining  \
0      FAP0007                   831         22             32901   
1      FAP0009                   831          0             27915   
2      FAP0010                   831        119             22098   
3      FAP0008                   892          0             14962   
4      FAP0001                   774        200             10318   
5      FAP0002                     0        634             10318   
6      FAP0003                     0        860             10318   
7      FAP0004                     0        924             10318   
8      FAP0005                     0        538             10318   
9      FAP0006                     0        965             10318   

   Part_2 Remaining  Part_3 Remaining  Part_4 Remaining  Part_5 Remaining  \
0             27397             26745             27320             25420   
1             19918             21759             19841             17941   
2        